# WNTR Basic Tutorial
The following tutorial illustrates basic use of WNTR, including use of the `WaterNetworkModel` object, the ability to read/write model files to other formats, run hydraulic and water quality simulations, compute resilience metrics, define and use fragility curves, skeletonize water network models, identify network segments associated with isolation valves, and assign geospatial data to junctions and pipes.

## Imports
Import WNTR and additional Python packages that are needed for the tutorial
- Numpy is required to define comparison operators (i.e., np.greater) in queries
- Scipy is required to define lognormal fragility curves
- NetworkX is used to compute topographic metrics
- Geopandas is used to load geospatial data
- Matplotlib is required to create graphics

In [ ]:
import numpy as np
from scipy.stats import lognorm
import networkx as nx
import geopandas as gpd
import matplotlib.pylab as plt
import wntr

## Units
WNTR uses SI (International System) units (length in meters, time in seconds, mass in kilograms).  See https://usepa.github.io/WNTR/units.html for more details.

# Water Network Model

The `WaterNetworkModel` object defines the water distribution system and simulation options. The object can be created from an EPANET INP file.

In [ ]:
# Create a WaterNetworkModel from an EPANET INP file
wn = wntr.network.WaterNetworkModel('../networks/Net3.inp')

In [ ]:
# Print a basic description of the model.  The level can be 0, 1, or 2 and defines the level of detail included in the description.
wn.describe(level=1)

In [ ]:
# List properties and methods associated with the WaterNetworkModel (omitting private underscore names)
[name for name in dir(wn) if not name.startswith('_')]

In [ ]:
# Plot a basic network graphic
ax = wntr.graphics.plot_network(wn)

## Nodes
Nodes define junctions, tanks, and reservoirs

In [ ]:
# Print the names of all junctions, tanks, and reservoirs
print("Node names", wn.node_name_list)

In [ ]:
# Print the names of just tanks
print("Tank names", wn.tank_name_list)

In [ ]:
# Get a tank object
tank = wn.get_node('1')
print(type(tank))
tank

In [ ]:
# List properties and methods associated with the tank (omitting private underscore names)
[name for name in dir(tank) if not name.startswith('_')]

In [ ]:
# Change the max level of a tank
print("Original max level", tank.max_level)
tank.max_level = 10
print("New max level", tank.max_level)

In [ ]:
# Add a junction to the WaterNetworkModel
wn.add_junction('new_junction', base_demand=0.0, demand_pattern=None, elevation=0.0, coordinates=None, demand_category=None)
print(wn.junction_name_list)

In [ ]:
# Remove a junction from the WaterNetworkModel
wn.remove_node('new_junction')
print(wn.junction_name_list)

## Links
Links define pipes, pumps, and valves

In [ ]:
# Print the names of all pipes, pumps, and valves
print("Link names", wn.link_name_list)

In [ ]:
# Print the names of just head pumps
print("Head pump names", wn.head_pump_name_list)

In [ ]:
# Get the name of links connected to a specific node
connected_links = wn.get_links_for_node('229')
print('Links connected to node 229 =', connected_links)

In [ ]:
# Get a pipe object
pipe = wn.get_link('105')
print(type(pipe))
pipe

In [ ]:
# List properties and methods associated with the pipe (omitting private underscore names)
[name for name in dir(pipe) if not name.startswith('_')]

In [ ]:
# Change the diameter of a pipe
print("Original diameter", pipe.diameter)
pipe.diameter = 10
print("New diameter", pipe.diameter)

In [ ]:
# Add a pipe to the WaterNetworkModel
wn.add_pipe(name="new_pipe", start_node_name="10", end_node_name="123", length=304.8, diameter=0.3048, roughness=100, minor_loss=0.0, initial_status='OPEN', check_valve=False)
print(wn.pipe_name_list)

In [ ]:
# Remove a pipe from the WaterNetworkModel
wn.remove_link("new_pipe")
print(wn.pipe_name_list)

## Demands and Patterns
Junctions can have multiple demands which are stored as Timeseries objects in a `demand_timeseries_list`. Each Timeseries contains a base value, pattern, and category.  Patterns contain multipliers and the pattern timestep.  

The following example illustrates how to
* Compute expected demand (which accounts for base demand, demand patterns, and demand multiplier)
* Compute average expected demand (average value for a 24 hour period -  accounts for base demand, demand patterns, and demand multiplier)
* Add demands to a junction
* Modify demand base value and pattern
* Remove demands from a junction
* Plot expected and simulated demands

In [ ]:
# Compute expected demand
expected_demand = wntr.metrics.expected_demand(wn)
expected_demand.head()

In [ ]:
# Compute and plot average expected demand 
AED = wntr.metrics.average_expected_demand(wn)
print(AED.head())
ax = wntr.graphics.plot_network(wn, node_attribute=AED, node_range=(0,0.025), title='Average expected demand (m$^3$/s)')

In [ ]:
# Identify junctions with zero demand
zero_demand = AED[AED == 0].index
print(zero_demand)
ax = wntr.graphics.plot_network(wn, node_attribute=list(zero_demand), title='Zero demand junctions')

In [ ]:
# Get the demands on Junction 15
junction = wn.get_node('15')
junction.demand_timeseries_list

In [ ]:
# Get the pattern associated with the demand
pattern = wn.get_pattern(junction.demand_timeseries_list[0].pattern_name)
pattern

In [ ]:
# Modify the base value of the demand
junction.demand_timeseries_list[0].base_value = 0.005

# Add a new pattern to the model
wn.add_pattern('New', [1,1,1,0,0,0,1,0,0.5,0.5,0.5,1])

# Use the new pattern to modify the junction demand
junction.demand_timeseries_list[0].pattern_name = "New"
print(junction.demand_timeseries_list)

In [ ]:
# Add a demand to Junction 15
junction.add_demand(base=0.015, pattern_name='1')
print(junction.demand_timeseries_list)

In [ ]:
# Plot original and modified expected demands
new_expected_demand = wntr.metrics.expected_demand(wn) 

plt.figure()
ax = expected_demand.loc[0:48*3600, "15"].plot()
new_expected_demand.loc[0:48*3600, "15"].plot(ax=ax)
tmp = ax.set_xlabel('Time (s)')
tmp = ax.set_ylabel('Expected demand (m$^3$/s)')

## Curves
Curves define pump head curves, tank volume curves, and pump efficiency curves.  The following example illustrates how to work with pump head curves and tank volume curves.

In [ ]:
# Get a head pump object and plot the head pump curve
pump = wn.get_link('10')
print(type(pump))
ax = wntr.graphics.plot_pump_curve(pump)

In [ ]:
# Get the head curve and print the points
pump_curve_name = pump.pump_curve_name
curve = wn.get_curve(pump_curve_name)
curve.points

In [ ]:
# Modify the curve points and re-plot the pump curve
curve.points = [(0.10, 20)]
ax = wntr.graphics.plot_pump_curve(pump)

In [ ]:
# Add a tank volume curve to the model and assign it to a tank
wn.add_curve('new_tank_curve', 'VOLUME', [
   (1,  0),
   (2,  60),
   (3,  188),
   (4,  372),
   (5,  596),
   (6,  848),
   (7,  1114),
   (8,  1379),
   (9,  1631),
   (10, 1856),
   (11, 2039),
   (12, 2168),
   (13, 2228)])
tank = wn.get_node('2')
tank.vol_curve_name = 'new_tank_curve'
ax = wntr.graphics.plot_tank_volume_curve(tank)

## Controls

Controls define conditions and actions that operate pipes, pumps, and valves.  WNTR includes support for EPANET controls and rules (note that both are stored as WNTR controls). As with EPANET, controls are evaluated after each simulation timestep, while rules are evaluated after each rule timestep (see `wn.options.time`). The method `convert_controls_to_rules` can be used to convert controls to rules, which can help avoid unintended behavior when controls and rules are both used in complex simulations.

In [ ]:
# Get a list of control names
wn.control_name_list

In [ ]:
# Print all controls
for name, controls in wn.controls():
    print(name, controls)

In [ ]:
# Get a specific control object
control = wn.get_control('control 18')
print(control)

In [ ]:
# Modify the control priority
control.update_priority(1)
print(control)

In [ ]:
# Add a time based pump control
pump = wn.get_link('10')
action = wntr.network.controls.ControlAction(pump, 'status', 1)
condition = wntr.network.controls.SimTimeCondition(wn, '=', '121:00:00')
control = wntr.network.controls.Control(condition, action, name='new_control')
wn.add_control('NewControl', control)
print(wn.control_name_list)

In [ ]:
# Remove a control
wn.remove_control('NewControl')
print(wn.control_name_list)

In [ ]:
# Convert controls to rules.  This can help avoid unintended behavior when controls and rules are both used in complex simulations.
wn.convert_controls_to_rules()

## Queries
Queries return attributes of nodes and links.  Comparison operations (like >, =) can be used to return a subset of attributes that meet specific criteria.

In [ ]:
# Return all pipe diameters (no comparison operator used in the query) 
all_pipe_diameters = wn.query_link_attribute('diameter')
all_pipe_diameters.head()

In [ ]:
# Return pipes diameters > 12 inches
large_pipe_diameters = wn.query_link_attribute('diameter', np.greater, 12*0.0254)
print("Number of pipes:", len(all_pipe_diameters))
print("Number of pipes > 12 inches:", len(large_pipe_diameters))

In [ ]:
# Plot pipes diameters > 12 inches
ax = wntr.graphics.plot_network(wn, link_attribute=large_pipe_diameters, node_size=0, link_width=2, title="Pipes with diameter > 12 inches")

## Coordinates
Node coordinates can be obtained using a node query.  Node coordinates can also be modified using functions in `wntr.morph`

In [ ]:
# Get node coordinates
coords = wn.query_node_attribute('coordinates')
coords

In [ ]:
# Rotate node coordinates counterclockwise by 30 degrees
wn_rotated = wntr.morph.rotate_node_coordinates(wn, 30)
ax = wntr.graphics.plot_network(wn_rotated)

## Loops and generators
Loops and generators are commonly used to modify network components or run stochastic simulations

In [ ]:
# Loop over tank names and objects with a generator
for name, tank in wn.tanks():
    print("Max level for tank", name, "=", tank.max_level)

In [ ]:
# Loop over tank names and then get the associated tank object
for name in wn.tank_name_list:
    tank = wn.get_node(name)
    print("Max level for tank", name, "=", tank.max_level)

## Pipe breaks and leaks
Pipes can be split (adding 1 junction to the model) or broken (adding two junctions to the model) using the `split_pipe` and `break_pipe` functions.  While a split pipe retains the network connectivity, a broken pipe does not connect across the break.  By default these functions return a copy of the WaterNetworkModel.

In [ ]:
# Split pipe 123 and add a leak to the new node which starts at hour 2 and ends at hour 12
wn = wntr.morph.split_pipe(wn, pipe_name_to_split='123', new_pipe_name='123_B', new_junction_name='123_node')
leak_node = wn.get_node('123_node')
leak_node.add_leak(wn, area=0.05, start_time=2*3600, end_time=12*3600)

In [ ]:
# Break pipe 121
wn = wntr.morph.break_pipe(wn, pipe_name_to_split='121', new_pipe_name='121_B', 
                           new_junction_name_old_pipe='121_node', new_junction_name_new_pipe='121B_node')

# Model I/O
A `WaterNetworkModel` can be converted to and from the following data formats and file types: 
* EPANET INP file
* dictionary
* JSON file
* NetworkX graph
* Geopandas GeoDataFrame
* GeoJSON and Shapefile file

In [ ]:
# Create a WaterNetworkModel from an EPANET INP file
wn = wntr.network.WaterNetworkModel('../networks/Net3.inp')

## EPANET INP files
WaterNetworkModel objects are commonly built from EPANET INP files.  WaterNetworkModel objects can also be saved as an EPANET INP file. Note that model attributes that are not EPANET compatible will not be saved in the INP file (i.e., leak attributes).

In [ ]:
# Create an EPANET INP file from a WaterNetworkModel
wntr.network.write_inpfile(wn, 'Net3_LPS.inp', units='LPS')

# Create a WaterNetworkModel from an EPANET INP file.  Note, this is equivalent to running `wn = wntr.network.WaterNetworkModel('Net3_LPS.inp')`
wn2 = wntr.network.read_inpfile('Net3_LPS.inp') 

## Dictionaries
Dictionaries offer a convenient Python format to store all the information in a WaterNetworkModel object. The dictionary can be saved to a file (typically a JSON file) to save the model. Unlike an EPANET INP file, dictionaries can contain custom model attributes.

In [ ]:
# Convert the WaterNetworkModel to a dictionary
wn_dict = wn.to_dict()
print(wn_dict.keys())

# Create a WaterNetworkModel from a dictionary
wn2 = wntr.network.from_dict(wn_dict)

## JSON files
JSON files can be created directly from a WaterNetworkModel object and hold the same information as the dictionary representation.

In [ ]:
# Create a JSON file from the WaterNetworkModel
wntr.network.write_json(wn, 'Net3.json')

# Create a WaterNetworkModel from a JSON file
wn2 = wntr.network.read_json('Net3.json')

## NetworkX graphs
Graphs facilitate topographic analysis using NetworkX. WaterNetworkModel objects are represented as a MultiDiGraph, which can have multiple edges between nodes and are directed (from start node to end node). Note that WaterNetworkModel objects cannot currently be created from NetworkX graphs.

In [ ]:
# Convert the WaterNetworkModel to a MultiDiGraph
G = wntr.network.to_graph(wn)
print(G)

## GeoPandas GeoDataFrames
GeoDataFrames store network attributes and geospatial geometry of junctions, tanks, reservoirs, pipes, pumps, and valves.  GeoDataFrames can be used in geospatial analysis, such as spap and intersection with other geospatial data. Note that WaterNetworkModels created from a collection of GeoDataFrames will not contain patterns, curves, rules, controls, or sources. The GeoDataFrames can be saved to file (typically GeoJSON or Shapefile) and loaded into GIS software platforms for further analysis.

In [ ]:
# Convert the WaterNetworkModel to a collection of GeoDataFrames
wn_gis = wntr.network.to_gis(wn)
print(wn_gis.junctions.head())
#print(wn_gis.tanks.head())
#print(wn_gis.reservoirs.head())
#print(wn_gis.pipes.head())
#print(wn_gis.pumps.head())
#print(wn_gis.valves.head())

# Create a WaterNetworkModel from a collection of GeoDataFrames
wn2 = wntr.network.from_gis(wn_gis)

## GeoJSON files and ESRI Shapefile files
GeoJSON and ESRI Shapefile files can be created directly from a WaterNetworkModel object.  The files can be loaded into GIS software platforms for further analysis. Note that column names longer than 10 characters will be truncated when saved to Shapefile. 

WaterNetworkModels can also be created from GeoJSON files or Shapefiles. A specific set of column names are required to define junctions, tanks, reservoirs, pipes, pumps, and valves (see the use of `valid_gis_names` below). Model attributes including controls, patterns, curves, and options need to be added separately.

In [ ]:
# Create GeoJSON files from the WaterNetworkModel
wntr.network.write_geojson(wn, 'Net3')

# Create a WaterNetworkModel from GeoJSON files
geojson_files = {'junctions': 'Net3_junctions.geojson',
                 'tanks': 'Net3_tanks.geojson',
                 'reservoirs': 'Net3_reservoirs.geojson',
                 'pipes': 'Net3_pipes.geojson',
                 'pumps': 'Net3_pumps.geojson'}
wn2 = wntr.network.read_geojson(geojson_files)

In [ ]:
# Compare model attributes of the original model with the model built from Shapefiles (note the absence of patterns and controls)
print(wn.describe(level=1))
print(wn2.describe(level=1))

In [ ]:
# Create Shapefiles from the WaterNetworkModel.  The following code will produce UserWarnings: "Column names longer than 10 characters will be truncated when saved to ESRI Shapefile."
wntr.network.write_shapefile(wn, 'Net3')

# Create a WaterNetworkModel from Shapefiles
shapefile_dirs = {'junctions': 'Net3_junctions',
                  'tanks': 'Net3_tanks',
                  'reservoirs': 'Net3_reservoirs',
                  'pipes': 'Net3_pipes',
                  'pumps': 'Net3_pumps'}
wn2 = wntr.network.read_shapefile(shapefile_dirs)

In [ ]:
# Compare model attributes of the original model with the model built from Shapefiles (note the absence of patterns and controls)
print(wn.describe(level=1))
print(wn2.describe(level=1))

In [ ]:
# Print valid GeoJSON or Shapefiles column names required to build a model
column_names = wntr.network.io.valid_gis_names()
print("Junction column names", column_names['junctions'])
print()
print("Tank column names", column_names['tanks'])
print()
print("Reservoir column names", column_names['reservoirs'])
print()
print("Pipe column names", column_names['pipes'])
print()
print("Pump column names", column_names['pumps'])
print()
print("Valve column names", column_names['valves'])

# Hydraulic and Water Quality Simulations

WNTR includes two simulators: the `EpanetSimulator` and the `WNTRSimulator`.  Both include the ability to run pressure dependent demand (PDD) or demand-driven (DD) hydraulic simulation.  Only the EpanetSimulator runs water quality simulations.

In [ ]:
# Create a WaterNetworkModel from an EPANET INP file
wn = wntr.network.WaterNetworkModel('../networks/Net3.inp')

## Simulation options
Simulation options include options related to simulation time, hydraulics, water quality, reactions, energy calculations, reporting, graphics, and user/custom options.

In [ ]:
# Print the WaterNetworkModel options
wn.options

In [ ]:
# Change the simulation duration to 4 days
wn.options.time.duration = 4*24*3600 # seconds
print(wn.options.time)

In [ ]:
# Change the simulation to use pressure dependent hydraulic analysis
wn.options.hydraulic.demand_model = 'PDD'
wn.options.hydraulic.required_pressure = 20 # m
wn.options.hydraulic.minimum_pressure = 2 # m
print(wn.options.hydraulic)

## EPANET and WNTR Simulators

In [ ]:
# Simulate hydraulics using EPANET
sim = wntr.sim.EpanetSimulator(wn)
results_EPANET = sim.run_sim()

In [ ]:
# Simulate hydraulics using the WNTRSimulator
sim = wntr.sim.WNTRSimulator(wn)
results_WNTR = sim.run_sim()

## Simulation results
Simulation results are stored in an object which includes a dictionary of DataFrames for nodes and a dictionary of DataFrames for links.  Each DataFrame is indexed by time (in seconds) and the columns are node or link names.

In [ ]:
# Print available node results
results_EPANET.node.keys()

In [ ]:
# Print available link results
results_EPANET.link.keys()

In [ ]:
# View EpanetSimulator pressure results
results_EPANET.node['pressure'].head()

In [ ]:
# Compare EpanetSimulator and WNTRSimulator pressure results
diff = results_EPANET.node['pressure'] - results_WNTR.node['pressure']
ax = diff.max(axis=1).plot(title='Max difference in pressure')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Pressure difference (m)')

In [ ]:
# Plot timeseries of tank levels
tank_levels = results_EPANET.node['pressure'].loc[:,wn.tank_name_list]
ax = tank_levels.plot(title='Tank level')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Tank Level (m)')

In [ ]:
# Plot timeseries of pump flowrates
pump_flowrates = results_EPANET.link['flowrate'].loc[:,wn.pump_name_list]
ax = pump_flowrates.plot(title='Pump flowrate')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Pump flowrate (m$^3$/s)')

In [ ]:
# Plot pressure at hour 5 on the network
pressure_at_5hr = results_EPANET.node['pressure'].loc[5*3600, :]
ax = wntr.graphics.plot_network(wn, node_attribute=pressure_at_5hr, node_size=30, title='Pressure at 5 hours')

## Reset initial conditions
Reset initial values, including simulation time, tank head, reservoir head, pipe status, pump status, and valve status.  This is required when using the WNTRSimulator in multiple simulations. Note, the EPANETSimulator is automatically reset.

In [ ]:
wn.reset_initial_values()

# Resilience Metrics

WNTR includes a wide range of metrics that can be used to compute resilience and related properties.

In [ ]:
# Create a WaterNetworkModel from an EPANET INP file
wn = wntr.network.WaterNetworkModel('../networks/Net3.inp')

## Topographic
Topographic metrics describe the physical layout of the system.

In [ ]:
# Convert the WaterNetworkModel to a MultiDiGraph
G = wn.to_graph() # directed multigraph

In [ ]:
# Some topographic metrics require an undirected graph or a graph with a single edge between two nodes
uG = G.to_undirected() # undirected multigraph
sG = nx.Graph(uG) # undirected simple graph (single edge between two nodes)

In [ ]:
# Articulation points
articulation_points = list(nx.articulation_points(uG))
ax = wntr.graphics.plot_network(wn, node_attribute=articulation_points, title="Articulation points")

In [ ]:
# Compute betweenness centrality
betweenness_centrality = nx.betweenness_centrality(G)
ax = wntr.graphics.plot_network(wn, node_attribute=betweenness_centrality, title="Betweenness centrality")

## Hydraulic
Hydraulic metrics are based on flow, demand, and/or pressure

In [ ]:
# Run a pressure dependent hydraulic simulation
wn.options.hydraulic.demand_model = 'PDD'
wn.options.hydraulic.required_pressure = 50 # m, The required pressure is set to create a scenario where not all demands are met

sim = wntr.sim.EpanetSimulator(wn)
results = sim.run_sim()

In [ ]:
# Compute water service availability, defined as the ratio of delivered demand to the expected demand. 
expected_demand = wntr.metrics.expected_demand(wn)
demand = results.node['demand'].loc[:,wn.junction_name_list]
wsa = wntr.metrics.water_service_availability(expected_demand.sum(axis=0), demand.sum(axis=0))
ax = wntr.graphics.plot_network(wn, node_attribute=wsa, title='Water service availability')

## Water Quality
Water quality metrics are based on the concentration or water age.

In [ ]:
# Compute water age using the last 48 hours of a water quality simulation
wn.options.quality.parameter = 'AGE'
sim = wntr.sim.EpanetSimulator(wn)
results = sim.run_sim()

age = results.node['quality']
age_last_48h = age.loc[age.index[-1]-48*3600:age.index[-1]]
average_age = age_last_48h.mean()/3600 # convert to hours for the plot
ax = wntr.graphics.plot_network(wn, node_attribute=average_age, title="Average water age (hr)")

In [ ]:
# Compute the population that is impacted by water age greater than 24 hours
pop = wntr.metrics.population(wn)
threshold = 24 # hours
pop_impacted = wntr.metrics.population_impacted(pop, average_age, np.greater, threshold)
ax = wntr.graphics.plot_network(wn, node_attribute=pop_impacted, title="Population impacted by water age > 24 hours")

# Fragility Curves

Fragility curves define the probability of exceeding a damage state as a function of environmental condition.  Fragility curves are commonly used in earthquake analysis, but can be defined for other scenarios.

In [ ]:
# Create a WaterNetworkModel from an EPANET INP file
wn = wntr.network.WaterNetworkModel('../networks/Net3.inp')

In [ ]:
FC = wntr.scenario.FragilityCurve()
FC.add_state('Minor', 1, {'Default': lognorm(0.5,scale=0.3)})
FC.add_state('Major', 2, {'Default': lognorm(0.5,scale=0.7)})
ax = wntr.graphics.plot_fragility_curve(FC, xlabel='Peak Ground Acceleration (g)')

In [ ]:
# Model peak ground acceleration from a magnitude 6.5 earthquake, assigning values to pipes.
wn = wntr.morph.scale_node_coordinates(wn, 1000)
epicenter = (32000,15000) # x,y location
magnitude = 6.5 # Richter scale
depth = 10000 # m, shallow depth
earthquake = wntr.scenario.Earthquake(epicenter, magnitude, depth)
distance = earthquake.distance_to_epicenter(wn, element_type=wntr.network.Pipe)
pga = earthquake.pga_attenuation_model(distance)
ax = wntr.graphics.plot_network(wn, link_attribute=pga, node_size=0, link_width=2)

In [ ]:
# Sample the failure probability and damage states for each pipe
failure_probability = FC.cdf_probability(pga)
damage_state = FC.sample_damage_state(failure_probability)

In [ ]:
# Plot damage state (converted to numeric values) on the network
priority_map = FC.get_priority_map()
damage_value = damage_state.map(priority_map)
custom_cmp = wntr.graphics.custom_colormap(3, ['grey', 'royalblue', 'darkorange'])
ax = wntr.graphics.plot_network(wn, link_attribute=damage_value,
    node_size=0, link_width=2, link_cmap=custom_cmp,
    title='Damage state: 0=None, 1=Minor, 2=Major')

# Network Skeletonization

Network skeletonization reduces the size of a WaterNetworkModel with the goal of having minimal impact on hydraulics. 

The skeletonization process retains all tanks, reservoirs, valves, and pumps, along with all junctions and pipes that are associated with controls. Junction demands and demand patterns are retained in the skeletonized model. Merged pipes are assigned equivalent properties for diameter, length, and roughness to approximate the updated system behavior. Pipes that are less than or equal to a user-defined pipe diameter threshold are candidates for removal based on branch trimming, series and parallel pipe merge.

In [ ]:
# Create a WaterNetworkModel from an EPANET INP file
wn = wntr.network.WaterNetworkModel('../networks/Net6.inp')
wn.describe(level=1)

In [ ]:
# Skeletonize the network using a 12 inch pipe diameter threshold
skel_wn = wntr.morph.skeletonize(wn, 12*0.0254)
skel_wn.describe(level=1)

In [ ]:
# Plot the original and skeletonized networks
ax = wntr.graphics.plot_network(wn, node_size=0, title='Original')
ax = wntr.graphics.plot_network(skel_wn, node_size=0, title='Skeletonized')

In [ ]:
# Simulate hydraulics on the original and skeletonized models and plot average pressure
sim = wntr.sim.EpanetSimulator(wn)
results_original = sim.run_sim()

sim = wntr.sim.EpanetSimulator(skel_wn)
results_skel = sim.run_sim()

In [ ]:
# Plot average pressure at junctions that exist in both the original and skeletonized model
skel_junctions = skel_wn.junction_name_list
pressure_orig = results_original.node['pressure'].loc[:,skel_junctions]
pressure_skel = results_skel.node['pressure'].loc[:,skel_junctions]

ax = pressure_orig.mean(axis=1).plot(label='Original')
ax = pressure_skel.mean(axis=1).plot(ax=ax, label='Skeletonized')
plt.title('Average pressure')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Pressure (m)')
plt.legend()

# Valve Segmentation

Valve segmentation groups links and nodes into segments based on the location of isolation valves. Unlike valves that are part of a WaterNetworkModel and used in hydraulic simulations (i.e. PRV, FCV), isolation valves are not included in the WaterNetworkModel and are defined as a separate data layer.

In [ ]:
# Create a WaterNetworkModel from an EPANET INP file
wn = wntr.network.WaterNetworkModel('../networks/Net3.inp')

In [ ]:
# Create a N-2 strategic valve layer.  Note that the user can create strategic or random valve placements, or use real valve data.
valve_layer = wntr.network.generate_valve_layer(wn, 'strategic', 2)
ax = wntr.graphics.plot_valve_layer(wn, valve_layer, add_colorbar=False)

In [ ]:
# Identify nodes and links that are in each valve segment
G = wn.to_graph()
node_segments, link_segments, seg_sizes = wntr.metrics.topographic.valve_segments(G, valve_layer)
seg_sizes.head()

In [ ]:
# Plot segments
N = seg_sizes.shape[0] # number of segments
cmap = wntr.graphics.random_colormap(N) # random color map helps visualize segments
ax = wntr.graphics.plot_network(wn, link_attribute=link_segments, node_size=0, link_width=2, link_range=[0,N],  link_cmap=cmap, title='Valve segment ID')

# Geospatial Capabilities
Geospatial data can be used within WNTR to build a WaterNetworkModel, associate geospatial data with nodes and links, and save simulation results to GIS compatible files.

**Note,  this example assumes the coordinate reference system (CRS) for the WaterNetworkModel and GIS data are in EPSG:4326 (lat/long). Geographic CRS are not suitable for measuring distance and the example will result in UserWarnings.  The example will be updated to use a projected CRS at a later date.**

In [ ]:
# Create a WaterNetworkModel from an EPANET INP file
wn = wntr.network.WaterNetworkModel('../networks/Net1.inp')

In [ ]:
# Convert a WaterNetworkModel to a collection of GeoDataFrames in EPSG:4326 coordinates
wn_gis = wntr.network.to_gis(wn, crs='EPSG:4326')
wn_gis.pipes.head()

In [ ]:
# Load hydrant data
hydrant_data = gpd.read_file('../data/Net1_hydrant_data.geojson') 
print(hydrant_data.crs)
hydrant_data.head()

## Snap data
The `snap` function is used to find the nearest point or line to a set of points. 

In [ ]:
# Snap hydrants to junctions
snapped_to_junctions = wntr.gis.snap(hydrant_data, wn_gis.junctions, tolerance=5.0)
snapped_to_junctions.head()

In [ ]:
# Plot results
ax = hydrant_data.plot()
ax.set_aspect('equal', adjustable='box')
ax = wntr.graphics.plot_network(wn, node_attribute=snapped_to_junctions['node'].to_list(), ax=ax)

## Intersect data
The `intersect` function is used to find the intersection between geometries.

In [ ]:
# Load demographic data associated with census block groups
demographic_data = gpd.read_file('../data/Net1_demographic_data.geojson') 
demographic_data.head()

In [ ]:
# Intersect junctions with census block groups, extract mean income
junction_demographics = wntr.gis.intersect(wn_gis.junctions, demographic_data, 'mean_income')
junction_demographics.head()

In [ ]:
# Intersect pipes with census block groups, extract mean income
pipe_demographics = wntr.gis.intersect(wn_gis.pipes, demographic_data, 'mean_income')
pipe_demographics.head()

In [ ]:
# Plot results
ax = demographic_data.plot(column='mean_income', alpha=0.5, cmap='bone', vmin=10000, vmax=100000)
ax.set_aspect('equal', adjustable='box')
ax = wntr.graphics.plot_network(wn, 
                                node_attribute=junction_demographics['mean'], 
                                link_attribute=pipe_demographics['weighted_mean'], 
                                link_width=1.5, ax=ax, title='Mean income ($)')

## Write analysis results to GIS files
In addition to the node and link attributes stored in the `WaterNetworkGIS` object, additional analysis results can be stored and saved to GIS compatible files.

In [ ]:
# Compute betweenness centrality and add it to the WaterNetworkGIS object
G = wn.to_graph() # directed multigraph
betweenness_centrality = nx.betweenness_centrality(G)

wn_gis.add_node_attributes(betweenness_centrality, 'betweenness_centrality')
wn_gis.junctions.head()

In [ ]:
# Run a hydraulic simulation and store average flowrate to the WaterNetworkGIS object
sim = wntr.sim.EpanetSimulator(wn)
results = sim.run_sim()
flowrate = results.link['flowrate'].mean()

wn_gis.add_link_attributes(flowrate, 'flowrate')
wn_gis.pipes.head()

In [ ]:
# Write the model and analysis results to GIS compatible files.  These files can be loaded into GIS platforms for further analysis.
wn_gis.write_geojson('Net3_analysis')